In [ ]:
!pip -q install ../input/pytorchtabnet/pytorch_tabnet-3.1.1-py3-none-any.whl
!pip -q install ../input/talib-binary/talib_binary-0.4.19-cp37-cp37m-manylinux1_x86_64.whl

In [ ]:
import numpy as np
import pandas as pd
import os, gc
from scipy.stats import pearsonr as p
import pickle as pkl
import warnings
from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor
from pytorch_tabnet.pretraining import TabNetPretrainer
import random
import torch
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts
from scipy.stats import pearsonr as p
warnings.simplefilter('ignore')

random.seed(0)
np.random.seed(0)

In [ ]:
df = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')

df.sort_values(by='time_id', inplace=True)


#split the dataset into train and test set with roughly 9:1 ratio

train_df = df[:2828086]
test_df = df[2828086:]



del df
gc.collect()



In [ ]:
X = train_df.drop(['row_id', 'target', 'time_id', 'investment_id'], axis = 1).values
y = train_df['target'].values.reshape(-1,1)

params = dict(
        n_d = 8,
        n_a = 8,
        n_steps = 3,
        gamma =1.3,
        n_independent = 2,
        n_shared = 2,
        lambda_sparse = 1e-3,
        optimizer_fn = Adam,
        optimizer_params = dict(lr = (0.024907164557092944)),
        mask_type = "entmax",
        scheduler_params = dict(T_0=200, T_mult=1, eta_min=1e-4, last_epoch=-1, verbose=False),
        scheduler_fn = CosineAnnealingWarmRestarts,
        seed = 0
    )  

In [ ]:
def train(params, X, y):
    
    model =  TabNetRegressor(**params)
    
    model.fit(
          X, y,
          max_epochs = 3,
          batch_size = 256,
          virtual_batch_size = 32,
          num_workers = 4,
          patience = 0
          )
    
    preds = model.predict(X)
    
    print('train score: ',  p(preds.flatten(), train_df.target)[0])
    
    return model
    
    
    

In [ ]:
def test(model, test_df):
    
    preds = model.predict(test_df.drop(['row_id', 'target', 'time_id', 'investment_id'], axis = 1).values)
    
    print('test score: ',  p(preds.flatten(), test_df.target)[0])
    

In [ ]:
model = train(params, X, y)

In [ ]:
test(model, test_df)